# Flatten tiles

# table of content
1) [Removing noise - hashing version](#removing-noise---hashing-version)
2) [Removing noise - points ID version](#removing-noise---points-id-version)
3) [Attribute points to closest instance](#attribute-points-to-closest-instance)

### Dependencies and general utils

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import open3d as o3d
import laspy
# import pdal
import json
import scipy
import copy
import pickle
from tqdm import tqdm
from scipy.spatial import cKDTree

### Utils

In [26]:
def remove_duplicates(laz_file, decimals=2):
    """
    Removes duplicate points from a LAS/LAZ file based on rounded 3D coordinates.

    Args:
        - laz_file (laspy.LasData): Input LAS/LAZ file as a laspy object.
        - decimals (int, optional): Number of decimals to round the coordinates for duplicate detection. Defaults to 2.

    Returns:
        - laspy.LasData: A new laspy object with duplicate points removed.
    """
        
    coords = np.round(np.vstack((laz_file.x, laz_file.y, laz_file.z)).T, decimals)
    _, unique_indices = np.unique(coords, axis=0, return_index=True)
    mask = np.zeros(len(coords), dtype=bool)
    mask[unique_indices] = True

    # Create new LAS object
    header = laspy.LasHeader(point_format=laz_file.header.point_format, version=laz_file.header.version)
    new_las = laspy.LasData(header)

    for dim in laz_file.point_format.dimension_names:
        setattr(new_las, dim, getattr(laz_file, dim)[mask])

    return new_las

def match_pointclouds(laz1, laz2):
    """Sort laz2 to match the order of laz1 without changing laz1's order.

    Args:
        laz1: laspy.LasData object (reference order)
        laz2: laspy.LasData object (to be sorted)
    
    Returns:
        laz2 sorted to match laz1
    """
    # Retrieve and round coordinates for robust matching
    coords_1 = np.round(np.vstack((laz1.x, laz1.y, laz1.z)), 2).T
    coords_2 = np.round(np.vstack((laz2.x, laz2.y, laz2.z)), 2).T

    # Verify laz2 is of the same size as laz1
    assert len(coords_2) == len(coords_1), "laz2 should be a subset of laz1"

    # Create a dictionary mapping from coordinates to indices
    coord_to_idx = {tuple(coord): idx for idx, coord in enumerate(coords_1)}

    # Find indices in laz1 that correspond to laz2
    matching_indices = []
    failed = 0
    for coord in coords_2:
        try:
            matching_indices.append(coord_to_idx[tuple(coord)])
        except Exception as e:
            failed += 1

    matching_indices = np.array([coord_to_idx[tuple(coord)] for coord in coords_2])

    # Sort laz2 to match laz1
    sorted_indices = np.argsort(matching_indices)

    # Apply sorting to all attributes of laz2
    laz2.points = laz2.points[sorted_indices]

    return laz2  # Now sorted to match laz1


def flattening_tile(tile_src, tile_new_original_src, grid_size=10, verbose=True):
    """
    Flattens a tile by interpolating the ground surface and subtracting it from the original elevation.

    Args:
        - tile_src (str): Path to the input tile in LAS/LAZ format.
        - tile_new_original_src (str): Path to save the resized original tile after filtering.
        - grid_size (int, optional): Size of the grid in meters for local interpolation. Defaults to 10.
        - verbose (bool, optional): Whether to display progress and debug information. Defaults to True.

    Returns:
        - None: Saves the floor and flattened versions of the tile and updates the original file.
    """

    # Load file
    laz = laspy.read(tile_src)
    init_len = len(laz)
    # laz = remove_duplicates(laz)
    if verbose:
        print(f"Removing duplicates: From {init_len} to {len(laz)}")
    # laz.write(tile_new_original_src)
    
    points = np.vstack((laz.x, laz.y, laz.z)).T
    points_flatten = copy.deepcopy(points)
    points_interpolated = copy.deepcopy(points)

    # Divide into tiles and find local minimums
    x_min, y_min = np.min(points[:, :2], axis=0)
    x_max, y_max = np.max(points[:, :2], axis=0)

    x_bins = np.append(np.arange(x_min, x_max, grid_size), x_max)
    y_bins = np.append(np.arange(y_min, y_max, grid_size), y_max)

    grid = {i:{j:[] for j in range(y_bins.size - 1)} for i in range(x_bins.size -1)}
    for _, (px, py, pz) in tqdm(enumerate(points), total=len(points), desc="Creating grid", disable=verbose==False):
        xbin = np.clip(0, (px - x_min) // grid_size, x_bins.size - 2)
        ybin = np.clip(0, (py - y_min) // grid_size, y_bins.size - 2)
        try:
            grid[xbin][ybin].append((px, py, pz))
        except Exception as e:
            print(xbin)
            print(ybin)
            print(x_bins)
            print(y_bins)
            print(grid.keys())
            print(grid[0].keys())
            raise e


    # Create grid_min
    grid_used = np.zeros((x_bins.size - 1, y_bins.size - 1))
    lst_grid_min = []
    lst_grid_min_pos = []
    for x in grid.keys():
        for y in grid[x].keys():
            if np.array(grid[x][y]).shape[0] > 0:
                grid_used[x, y] = 1
                lst_grid_min.append(np.min(np.array(grid[x][y])[:,2]))
                arg_min = np.argmin(np.array(grid[x][y])[:,2])
                lst_grid_min_pos.append(np.array(grid[x][y])[arg_min,0:2])
            else:
                grid_used[x, y] = 0
    arr_grid_min_pos = np.vstack(lst_grid_min_pos)
    if verbose:
        print("Resulting grid:")
        print(arr_grid_min_pos.shape)
        print(grid_used)

    # Interpolate
    points_xy = np.array(points)[:,0:2]
    interpolated_min_z = scipy.interpolate.griddata(arr_grid_min_pos, np.array(lst_grid_min), points_xy, method="cubic")

    # Fill NaNs with nearest neighbor interpolation
    nan_mask = np.isnan(interpolated_min_z)
    x = np.array(points)[:,0]
    y = np.array(points)[:,1]
    z = np.array(points)[:,2]

    if np.any(nan_mask):
        interpolated_min_z[nan_mask] = scipy.interpolate.griddata(arr_grid_min_pos, np.array(lst_grid_min), (x[nan_mask], y[nan_mask]), method='nearest')

    mask_valid = np.array([x != -1 for x in list(interpolated_min_z)])
    points_interpolated = points_interpolated[mask_valid]
    points_interpolated[:, 2] = interpolated_min_z[mask_valid]

    if verbose:
        print("Interpolation:")
        print(f"Original number of points: {points.shape[0]}")
        print(f"Interpollated number of points: {points_interpolated.shape[0]} ({int(points_interpolated.shape[0] / points.shape[0]*100)}%)")

    # save floor
    filtered_points = {dim: getattr(laz, dim)[mask_valid] for dim in laz.point_format.dimension_names}
    header = laspy.LasHeader(point_format=laz.header.point_format, version=laz.header.version)
    new_las = laspy.LasData(header)

    #   _Assign filtered and modified data
    for dim, values in filtered_points.items():
        setattr(new_las, dim, values)
    setattr(new_las, 'x', points_interpolated[:,0])
    setattr(new_las, 'y', points_interpolated[:,1])
    setattr(new_las, 'z', points_interpolated[:,2])

    #   _Save new file
    floor_dir = os.path.join(os.path.dirname(tile_src), 'floor')
    os.makedirs(floor_dir, exist_ok=True)
    new_las.write(os.path.join(floor_dir, os.path.basename(tile_src).split('.laz')[0] + f"_floor_{grid_size}m.laz"))
    if verbose:
        print("Saved file: ", os.path.join(floor_dir, os.path.basename(tile_src).split('.laz')[0] + f"_floor_{grid_size}m.laz"))

    # Flatten
    points_flatten = points_flatten[mask_valid]
    points_flatten[:,2] = points_flatten[:,2] - points_interpolated[:,2]

    filtered_points = {dim: getattr(laz, dim)[mask_valid] for dim in laz.point_format.dimension_names}
    header = laspy.LasHeader(point_format=laz.header.point_format, version=laz.header.version)
    header.point_count = 0
    new_las = laspy.LasData(header)


    #   _Assign filtered and modified data
    for dim, values in filtered_points.items():
        setattr(new_las, dim, values)

    setattr(new_las, 'x', points_flatten[:,0])
    setattr(new_las, 'y', points_flatten[:,1])
    setattr(new_las, 'z', points_flatten[:,2])

    #   _Save new file
    flatten_dir = os.path.join(os.path.dirname(tile_src), 'flatten')
    os.makedirs(flatten_dir, exist_ok=True)
    new_las.write(os.path.join(flatten_dir, os.path.basename(tile_src).split('.laz')[0] + f"_flatten_{grid_size}m.laz"))
    if verbose:
        print("Saved file: ", os.path.join(flatten_dir, os.path.basename(tile_src).split('.laz')[0] + f"_flatten_{grid_size}m.laz"))

    # Resize original file
    laz.points = laz.points[mask_valid]
    laz.write(tile_new_original_src)
    if verbose:
        print("Saved file: ", tile_new_original_src)


def flattening(src_tiles, src_new_tiles, grid_size=10, verbose=True, verbose_full=False):
    """
    Applies the flattening process to all tiles in a directory using grid-based ground surface estimation.

    Args:
        - src_tiles (str): Path to the directory containing original tiles.
        - src_new_tiles (str): Path to the directory where resized tiles will be saved.
        - grid_size (int, optional): Size of the grid in meters for interpolation. Defaults to 10.
        - verbose (bool, optional): Whether to show a general progress bar. Defaults to True.
        - verbose_full (bool, optional): Whether to print detailed info per tile. Defaults to False.

    Returns:
        - None: Processes and saves flattened tiles into their respective folders.
    """
    
    print("Starting flattening:")
    list_tiles = [x for x in os.listdir(src_tiles) if x.endswith('.laz')]
    for _, tile in tqdm(enumerate(list_tiles), total=len(list_tiles), desc="Processing", disable=verbose==False):
        if verbose_full:
            print("Flattening tile: ", tile)
        flattening_tile(
            tile_src=os.path.join(src_tiles, tile), 
            tile_new_original_src=os.path.join(src_new_tiles, tile),
            grid_size=grid_size,
            verbose=verbose_full,
            )

### Removing noise - hashing version

In [16]:
src_tile_test = r"D:\GitHubProjects\PointCloudUtils\data\test_vegetation_removal\SwissSurface_2019_2538_1154_tile_21.laz"
grid_size = 1
src_tile_test_new = src_tile_test.split('.laz')[0] + f"_new_grid_size={grid_size}m.laz"

In [17]:
# flatten
flattening_tile(src_tile_test, src_tile_test_new, grid_size)

Removing duplicates: From 298527 to 298527


Creating grid: 100%|██████████| 298527/298527 [00:05<00:00, 56048.64it/s]


Resulting grid:
(10000, 2)
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
Interpolation:
Original number of points: 298527
Interpollated number of points: 298527 (100%)
Saved file:  D:\GitHubProjects\PointCloudUtils\data\test_vegetation_removal\floor\SwissSurface_2019_2538_1154_tile_21_floor_1m.laz
Saved file:  D:\GitHubProjects\PointCloudUtils\data\test_vegetation_removal\flatten\SwissSurface_2019_2538_1154_tile_21_flatten_1m.laz
Saved file:  D:\GitHubProjects\PointCloudUtils\data\test_vegetation_removal\SwissSurface_2019_2538_1154_tile_21_new_grid_size=1m.laz


#### add samples as vegetation in orginal

In [32]:
src_foler_instances_for_regrouping = r"D:\GitHubProjects\PointCloudUtils\data\test_vegetation_removal\flatten\SwissSurface_2019_2538_1154_tile_21_flatten_1m_stripes_100_10"
src_laz = r"D:\GitHubProjects\PointCloudUtils\data\test_vegetation_removal\flatten\SwissSurface_2019_2538_1154_tile_21_flatten_1m.laz"

In [ ]:
def hash_coords(x, y, z, rounding=2):
    """Create a unique integer hash for each rounded coordinate triple."""
    return np.round(x, rounding) * 1e12 + np.round(y, rounding) * 1e6 + np.round(z, rounding)

list_instances_src = [x for x in os.listdir(src_foler_instances_for_regrouping) if x.endswith('.laz')]
tile = laspy.read(src_laz)
classification = getattr(tile, "classification")

# Precompute hashes for the full tile once
tile_hash = hash_coords(tile.x, tile.y, tile.z)

for id_instance, instance_src in tqdm(enumerate(list_instances_src), total=len(list_instances_src)):
    instance = laspy.read(os.path.join(src_foler_instances_for_regrouping, instance_src))
    instance_hash = hash_coords(instance.x, instance.y, instance.z)

    # Fast membership test using np.isin (vectorized)
    mask = np.isin(tile_hash, instance_hash)
    classification[mask] = 42

# Store results
setattr(tile, 'classification', classification)

# Save file
tile.write(os.path.join(src_laz.split('.laz')[0] + "_segmented.laz"))

100%|██████████| 10/10 [00:00<00:00, 25.81it/s]


#### copy classification of samples

In [36]:
def hash_coords(x, y, z, rounding=2):
    """Create a unique integer hash for each rounded coordinate triple."""
    return np.round(x, rounding) * 1e12 + np.round(y, rounding) * 1e6 + np.round(z, rounding)

list_instances_src = [x for x in os.listdir(src_foler_instances_for_regrouping) if x.endswith('.laz')]
tile = laspy.read(src_laz)
classification = getattr(tile, "classification")


# Precompute hashes for the full tile once
tile_hash = hash_coords(tile.x, tile.y, tile.z)

for id_instance, instance_src in tqdm(enumerate(list_instances_src), total=len(list_instances_src)):
    instance = laspy.read(os.path.join(src_foler_instances_for_regrouping, instance_src))
    instance_hash = hash_coords(instance.x, instance.y, instance.z)

    # Fast membership test using np.isin (vectorized)
    mask = np.isin(tile_hash, instance_hash)
    classification[mask] = getattr(instance, 'classification')

# Store results
setattr(tile, 'classification', classification)

# Save file
tile.write(os.path.join(src_laz.split('.laz')[0] + "_segmented.laz"))

100%|██████████| 10/10 [00:00<00:00, 25.49it/s]


#### Update classification file between two samples

In [37]:
src_original = r"D:\GitHubProjects\PointCloudUtils\data\test_vegetation_removal\SwissSurface_2019_2538_1154_tile_21.laz"
src_segmented = r"D:\GitHubProjects\PointCloudUtils\data\test_vegetation_removal\flatten\SwissSurface_2019_2538_1154_tile_21_flatten_1m_segmented.laz"

In [ ]:
laz_original = laspy.read(src_original)
laz_segmented = laspy.read(src_segmented)
field_name = 'classification'
src_result = src_original.split('.laz')[0] + '_segmented.laz'
setattr(laz_original, field_name, getattr(laz_segmented, field_name))
laz_original.write(src_result)

298527
298527


### Removing noise - points ID version

In [24]:
src_tile = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\lausanne\distributed\gt\clean\SwissSurface_2019_2538_1154_tile_23_gt\SwissSurface_2019_2538_1154_tile_23_gt.laz"
grid_size = 10
src_tile_new = src_tile.split('.laz')[0] + f"_new_grid_size={grid_size}m.laz"
src_tile_flatten = os.path.join(os.path.dirname(src_tile), 'flatten', os.path.basename(src_tile).split('.laz')[0] + f"_flatten_{grid_size}m.laz")

#### flatten

In [25]:
# flatten
flattening_tile(src_tile, src_tile_new, grid_size)

Removing duplicates: From 292326 to 292326


Creating grid:   0%|          | 0/292326 [00:00<?, ?it/s]

Creating grid: 100%|██████████| 292326/292326 [00:03<00:00, 85816.93it/s]


Resulting grid:
(100, 2)
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
Interpolation:
Original number of points: 292326
Interpollated number of points: 280522 (95%)
Saved file:  D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\lausanne\distributed\gt\clean\SwissSurface_2019_2538_1154_tile_23_gt\floor\SwissSurface_2019_2538_1154_tile_23_gt_floor_10m.laz
Saved file:  D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\lausanne\distributed\gt\clean\SwissSurface_2019_2538_1154_tile_23_gt\flatten\SwissSurface_2019_2538_1154_tile_23_gt_flatten_10m.laz
Saved file:  D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\lausanne\distributed\gt\clean\SwissSurface_2019_2538_1154_tile_23_gt\SwissSurface_2019_25

In [34]:
# load resulting original tile
tile_original = laspy.read(src_tile_new)
tile_flatten = laspy.read(src_tile_flatten)

# add id
if 'id' not in list(tile_original.point_format.dimension_names):
    tile_original.add_extra_dim(laspy.ExtraBytesParams('id',type="uint32"))
if 'id' not in list(tile_flatten.point_format.dimension_names):
    tile_flatten.add_extra_dim(laspy.ExtraBytesParams('id',type="uint32"))

# set id
setattr(tile_original, 'id', range(len(tile_original)))
setattr(tile_flatten, 'id', range(len(tile_original)))

src_tile_with_id = src_tile.split('.laz')[0] + "_with_id.laz"
src_tile_flatten_with_id = src_tile_flatten.split('.laz')[0] + "_with_id.laz"
tile_original.write(src_tile_with_id)
tile_flatten.write(src_tile_flatten_with_id)


In [ ]:
src_soil_clean = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\lausanne\distributed\gt\clean\SwissSurface_2019_2538_1154_tile_23_gt\flatten\SwissSurface_2019_2538_1154_tile_23_gt_flatten_10m_clean_ground.laz"
tile_soil_clean = laspy.read(src_soil_clean)
ids = getattr(tile_soil_clean, "id")

mask = np.zeros(len(tile_original), dtype=bool)
mask[ids] = True
mask_old_trees = tile_original.gt_semantic == 0
mask_noise = ~mask & mask_old_trees

getattr(tile_original, "gt_semantic")[:] = 1
getattr(tile_original, "gt_semantic")[mask] = 0
getattr(tile_original, "classification")[mask_noise] = 42

tile_original.write(os.path.join(src_tile.split('.laz')[0] + "_segmented.laz"))

### Attribute points to closest instance

In [28]:
# Setting
src_file = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\lausanne\distributed\gt\clean\SwissSurface_2019_2538_1154_tile_23_gt\SwissSurface_2019_2538_1154_tile_23_gt_segmented.laz"
attr_of_points = 'classification'
val_points = 42
attr_target_inst = 'gt_instance'
attr_target_sem = 'gt_semantic'

In [29]:
# Find centroids of instance
laz_file = laspy.read(src_file)
set_instances = set(getattr(laz_file, attr_target_inst))
instances = getattr(laz_file, attr_target_inst)
centroids_ids = []
centroids_pos = []
for inst_num in set_instances:
    if inst_num == 0:
        continue
    mask = instances == inst_num
    x_mean = np.mean(getattr(laz_file, 'x')[mask])
    y_mean = np.mean(getattr(laz_file, 'y')[mask])
    centroids_ids.append(inst_num)
    centroids_pos.append((x_mean, y_mean))
    # centroids[inst_num] = (x_mean, y_mean)
centroids_pos = np.asarray(centroids_pos)

In [30]:
# Attribute points
mask_points = getattr(laz_file, attr_of_points) == val_points
points = laz_file.xyz[mask_points]

corresponding_clusters = []

for _, point_id in tqdm(enumerate(range(len(points))), total=len(points)):
    point = np.array(points[point_id][0:-1])
    dists = np.sqrt(np.sum((centroids_pos - point)**2, axis=1))
    id_min = np.argmin(dists)
    corresponding_clusters.append(centroids_ids[id_min])
print(corresponding_clusters)


100%|██████████| 32190/32190 [00:00<00:00, 65186.55it/s]

[267, 267, 267, 266, 266, 267, 267, 267, 267, 267, 267, 267, 267, 267, 267, 128, 267, 267, 267, 267, 267, 267, 264, 128, 128, 128, 43, 128, 128, 128, 128, 128, 12, 43, 12, 12, 39, 39, 12, 12, 12, 12, 12, 128, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 73, 45, 73, 128, 128, 128, 128, 128, 268, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 73, 13, 13, 13, 45, 45, 128, 128, 271, 271, 271, 271, 268, 271, 271, 271, 271, 271, 271, 271, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 47, 45, 45, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 47, 13, 13, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 13, 13, 13, 271, 271, 271, 271, 

In [31]:
# set it to original file
# setattr(laz_file[mask_points], attr_target_inst, corresponding_clusters)
getattr(laz_file, attr_target_inst)[mask_points] = corresponding_clusters
getattr(laz_file, attr_target_sem)[mask_points] = 1
laz_file.write(src_file.split('.laz')[0] + '_noise_attributed.laz')

# Temp

In [3]:
laz = laspy.read(r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2024_AllScans_Subsample4cm.laz")
print(np.mean(getattr(laz, 'x')))
print(np.mean(getattr(laz, 'y')))
print(np.mean(getattr(laz, 'z')))

-505.88210236359214
-677.7981444825483
65.71110542231727


In [4]:
x_true = 2587613.523987
y_true = 1099147.691040
z_true = 1306.376953

x_shifted = -71.776970
y_shifted = -359.969265
z_shifted = -88.544250

x_translate = x_true - x_shifted
y_translate = y_true - y_shifted
z_translate = z_true - z_shifted

print(x_translate)
print(y_translate)
print(z_translate)

2587685.300957
1099507.660305
1394.9212029999999


In [11]:
print(getattr(laz,'x').shape)
print((getattr(laz,'x')+x_translate).shape)
print(getattr(laz,'x')[0])
print((getattr(laz,'x')+x_translate)[0])

(123499372,)
(123499372,)
-903.212
2586782.088957


In [69]:
src_temp = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2024_AllScans_Subsample4cm.laz"
src_target = src_temp.split('.laz')[0] + "_temp.laz"
laz_temp = laspy.read(src_temp)

In [70]:
new_x = np.zeros(len(laz_temp))
new_y = np.zeros(len(laz_temp))
new_z = np.zeros(len(laz_temp))
new_names = ['new_x', 'new_y', 'new_z']
old_names = ['x', 'y', 'z']
new_offset = []

for new_val, old_val, shift in zip(new_names, old_names, [x_translate, y_translate, z_translate]):
    # print(getattr(laz_temp, old_val))
    # print(shift)
    # print(int(shift*1000))
    # values = getattr(laz_temp, old_val) + shift
    # print(values)
    # print(np.round(values,3))
    # print(laz_temp.header.offsets)
    # print(laz_temp.header.scales)
    new_offset.append(np.mean(getattr(laz_temp, old_val)) + shift)
    
    # laz_temp.header.offsets = [int(x_translate), int(y_translate), int(z_translate)]
    # laz_temp.add_extra_dim(laspy.ExtraBytesParams(name=new_val, type=np.float32))  # Change type if needed
    # laz_temp[old_val] = np.round(values,3)
    # break
laz_temp.header.offsets = new_offset

In [71]:
print(list(laz_temp.point_format.dimension_names))
print(np.min(getattr(laz_temp, 'x')))
print(np.max(getattr(laz_temp, 'x')))

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'red', 'green', 'blue', 'Amplitude']
-1035.9370000000001
-29.402000000000044


In [72]:
# src_target = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2024_AllScans_Subsample4cm_temp.laz"
laz_temp.write(src_target)

In [28]:
# print(x_translate)
# print(type(getattr(laz, 'x')))
# print(laz.header.scales)
# # new_scale = (0.01, 0.01, 0.01)
# new_offset = (
#     np.min(laz.x + x_translate),
#     np.min(laz.y + y_translate),
#     np.min(laz.z + y_translate),
# )
# # laz_temp.header.scales = new_scale
# laz_temp.header.scales = new_offset
# laz_temp.write(r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2024_AllScans_Subsample4cm_translated.laz")
# # setattr(laz_temp, 'x', getattr(laz, 'x') + 10000000)
# # setattr(laz, 'y', getattr(laz, 'y') + y_translate)
# # setattr(laz, 'z', getattr(laz, 'z') + z_translate)

In [75]:
src_temp = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2024_AllScans_Subsample4cm_sub.laz"
src_target = src_temp.split('.laz')[0] + "_temp.laz"
# laz_temp = laspy.read(src_temp)
old = laspy.read(src_temp)

# Define shift
dx, dy, dz = 1e6, 2e6, 0

# Prepare new coordinates
x_new = old.x + dx
y_new = old.y + dy
z_new = old.z + dz

# Build a new header
hdr = laspy.LasHeader(point_format=old.header.point_format.id, version=old.header.version)
hdr.offsets = [np.min(x_new), np.min(y_new), np.min(z_new)]
hdr.scales = [0.001, 0.001, 0.001]

new = laspy.LasData(hdr)

# Copy all attributes
# for name in old.point_format.dimension_names:
#     if name in ["X", "Y", "Z"]:
#         continue
#     setattr(new, name, getattr(old, name))

# Assign new coordinates
new.x, new.y, new.z = x_new, y_new, z_new

new.write(src_target)